In [1]:

# # Laad de CSV-bestanden
# data_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
# train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
# label_coords_df = pd.read_csv(os.path.join(data_dir, 'train_label_coordinates.csv'))
# series_desc_df = pd.read_csv(os.path.join(data_dir, 'train_series_descriptions.csv'))

# # Combineer de gegevens op basis van study_id en series_id
# merged_df = label_coords_df.merge(series_desc_df, on=['study_id', 'series_id'])
# merged_df = merged_df.merge(train_df, on='study_id')

# # Voeg een kolom toe voor de image path
# merged_df['image_path'] = merged_df.apply(lambda row: os.path.join(data_dir, 'train_images', str(row['study_id']), str(row['series_id']), f"{row['instance_number']}.dcm"), axis=1)

# # Selecteer relevante kolommen voor classificatie (pas deze aan naar je eigen taak)
# conditions = [
#     'spinal_canal_stenosis_l1_l2', 'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4',
#     'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1',
#     'left_neural_foraminal_narrowing_l1_l2', 'left_neural_foraminal_narrowing_l2_l3',
#     'left_neural_foraminal_narrowing_l3_l4', 'left_neural_foraminal_narrowing_l4_l5'
# ]

# # Houd alleen de rijen waar alle condities beschikbaar zijn
# merged_df = merged_df.dropna(subset=conditions)

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pydicom
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm
import cv2
import glob
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import torchvision.transforms as transforms
import torchvision.utils as vutils
from sklearn.model_selection import KFold
import albumentations as A
import timm
import re

/opt/conda/lib/python3.10/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [3]:
# # List out all of the Studies we have on patients.
# part_1 = os.listdir('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images')
# part_1 = list(filter(lambda x: x.find('.DS') == -1, part_1))


# df_meta_f = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')

# p1 = [(x, f"/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/{x}") for x in part_1]
# meta_obj = { p[0]: { 'folder_path': p[1], 
#                     'SeriesInstanceUIDs': [] 
#                    } 
#             for p in p1 }
# for m in meta_obj:
#     meta_obj[m]['SeriesInstanceUIDs'] = list(
#         filter(lambda x: x.find('.DS') == -1, 
#                os.listdir(meta_obj[m]['folder_path'])
#               )
#     )


# # grabs the correspoding series descriptions
# for k in tqdm(meta_obj):
#     for s in meta_obj[k]['SeriesInstanceUIDs']:
#         if 'SeriesDescriptions' not in meta_obj[k]:
#             meta_obj[k]['SeriesDescriptions'] = []
#         try:
#             meta_obj[k]['SeriesDescriptions'].append(
#                 df_meta_f[(df_meta_f['study_id'] == int(k)) & 
#                 (df_meta_f['series_id'] == int(s))]['series_description'].iloc[0])
#         except:
#             print("Failed on", s, k)

# meta_obj[list(meta_obj.keys())[1]]
# patient = train_df.iloc[1]
# ptobj = meta_obj[str(patient['study_id'])]

In [4]:
# meta_obj[list(meta_obj.keys())[1]]

In [5]:
# patient = train_df.iloc[1]

In [6]:
# df_coor = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv')

In [7]:
# ptobj = meta_obj[str(patient['study_id'])]

In [8]:
# im_list_dcm = {}
# for idx, i in enumerate(ptobj['SeriesInstanceUIDs']):
#     im_list_dcm[i] = {'images': [], 'description': ptobj['SeriesDescriptions'][idx]}
#     images = glob.glob(f"{ptobj['folder_path']}/{ptobj['SeriesInstanceUIDs'][idx]}/*.dcm")
#     for j in sorted(images, key=lambda x: int(x.split('/')[-1].replace('.dcm', ''))):
#         im_list_dcm[i]['images'].append({
#             'SOPInstanceUID': j.split('/')[-1].replace('.dcm', ''), 
#             'dicom': pydicom.dcmread(j) })

In [9]:
def display_coor_on_img(c, i, title):
    center_coordinates = (int(c['x']), int(c['y']))
    radius = 10
    color = (255, 0, 0)  # Red color in BGR
    thickness = 2
    IMG = i['dicom'].pixel_array
    IMG_normalized = cv2.normalize(IMG, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    IMG_with_circle = cv2.circle(IMG_normalized.copy(), center_coordinates, radius, color, thickness)
    
    # Convert the image from BGR to RGB for correct color display in matplotlib
    IMG_with_circle = cv2.cvtColor(IMG_with_circle, cv2.COLOR_BGR2RGB)
    
    # Display the image
    plt.imshow(IMG_with_circle)
    plt.axis('off')  # Turn off axis numbers and ticks
    plt.title(title)
    plt.show()

In [10]:
# coor_entries = df_coor[df_coor['study_id'] == int(patient['study_id'])]

In [11]:
# print("Only showing severe cases for this patient")
# for idc, c in coor_entries.iterrows():
#     for i in im_list_dcm[str(c['series_id'])]['images']:
#         if int(i['SOPInstanceUID']) == int(c['instance_number']):
#             try:
#                 patient_severity = patient[
#                     f"{c['condition'].lower().replace(' ', '_')}_{c['level'].lower().replace('/', '_')}"
#                 ]
#             except Exception as e:
#                 patient_severity = "unknown severity"
#             title = f"{i['SOPInstanceUID']} \n{c['level']}, {c['condition']}: {patient_severity} \n{c['x']}, {c['y']}"
#             if patient_severity == 'Severe':
#                 display_coor_on_img(c, i, title)

In [12]:
# !pip install torchio open3d timm_3d spacecutter skorch

In [13]:
# import gc
# import sys
# import math, random
# import matplotlib.pyplot as plt

# import traceback
# from collections import OrderedDict

# import torch.nn.functional as F
# from torch import nn
# from torch.utils.data import DataLoader, Dataset
# from torch.optim import AdamW

# from transformers import get_cosine_schedule_with_warmup

In [14]:
# def imread_and_imwirte(src_path, dst_path):
#     dicom_data = pydicom.dcmread(src_path)
#     image = dicom_data.pixel_array
#     image = (image - image.min()) / (image.max() - image.min() +1e-6) * 255
#     img = cv2.resize(image, (512, 512),interpolation=cv2.INTER_CUBIC)
#     assert img.shape==(512,512)
#     cv2.imwrite(dst_path, img)

In [15]:
rd = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
df = pd.read_csv(f'{rd}/train_series_descriptions.csv')
st_ids = df['study_id'].unique()
dfc = pd.read_csv(f'{rd}/train_label_coordinates.csv')
desc = list(df['series_description'].unique())

In [16]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [17]:
# for idx, si in enumerate(tqdm(st_ids, total=len(st_ids))):
#     pdf = df[df['study_id'] == si]
#     for ds in desc:
#         ds_ = ds.replace('/', '_')
#         pdf_ = pdf[pdf['series_description'] == ds]
#         os.makedirs(f'cvt_png/{si}/{ds_}', exist_ok=True)
#         allimgs = []
#         for i, row in pdf_.iterrows():
#             pimgs = glob.glob(f'{rd}/train_images/{row["study_id"]}/{row["series_id"]}/*.dcm')
#             pimgs = sorted(pimgs, key=natural_keys)
#             allimgs.extend(pimgs)
            
#         if len(allimgs) == 0:
#             print(si, ds, 'has no images')
#             continue

            
#         if len(allimgs)==0:
#             print(si, ds, 'has no images')
#             print(pimgs)
#             print(allimgs)
#             continue

#         if ds == 'Axial T2':
#             for j, impath in enumerate(allimgs):
#                 dst = f'cvt_png/{si}/{ds}/{j:03d}.png'
#                 imread_and_imwirte(impath, dst)
                
#         elif ds == 'Sagittal T2/STIR':
            
#             step = len(allimgs) / 10.0
#             st = len(allimgs)/2.0 - 4.0*step
#             end = len(allimgs)+0.0001
#             for j, i in enumerate(np.arange(st, end, step)):
#                 dst = f'cvt_png/{si}/{ds_}/{j:03d}.png'
#                 ind2 = max(0, int((i-0.5001).round()))
#                 imread_and_imwirte(allimgs[ind2], dst)
                
#             assert len(glob.glob(f'cvt_png/{si}/{ds_}/*.png'))==10
                
#         elif ds == 'Sagittal T1':
#             step = len(allimgs) / 10.0
#             st = len(allimgs)/2.0 - 4.0*step
#             end = len(allimgs)+0.0001
#             for j, i in enumerate(np.arange(st, end, step)):
#                 dst = f'cvt_png/{si}/{ds}/{j:03d}.png'
#                 ind2 = max(0, int((i-0.5001).round()))
#                 imread_and_imwirte(allimgs[ind2], dst)
                
#             assert len(glob.glob(f'cvt_png/{si}/{ds}/*.png'))==10

In [18]:
NOT_DEBUG = True # True -> run naormally, False -> debug mode, with lesser computing cost

OUTPUT_DIR = f'rsna24-results'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
N_WORKERS = os.cpu_count() 
USE_AMP = True # can change True if using T4 or newer than Ampere
SEED = 8620
MODEL_DIR= "/kaggle/input/test/pytorch/default/2"
IMG_SIZE = [512, 512]
IN_CHANS = 30
N_LABELS = 25
N_CLASSES = 3 * N_LABELS

AUG_PROB = 0.75

N_FOLDS = 3 if NOT_DEBUG else 2
EPOCHS = 20 if NOT_DEBUG else 2
MODEL_NAME = "tf_efficientnet_b3.ns_jft_in1k" if NOT_DEBUG else "tf_efficientnet_b0.ns_jft_in1k"

GRAD_ACC = 2
TGT_BATCH_SIZE = 32
BATCH_SIZE = TGT_BATCH_SIZE // GRAD_ACC
MAX_GRAD_NORM = None
EARLY_STOPPING_EPOCH = 3

LR = 2e-4 * TGT_BATCH_SIZE / 32
WD = 1e-2
AUG = True


In [19]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [20]:
def set_random_seed(seed: int = 8620, deterministic: bool = False):
    """Set seeds"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = deterministic  # type: ignore


In [21]:
# df = pd.read_csv(f'{rd}/train.csv')
# df.head()

In [22]:
# df = df.fillna(-100)

In [23]:
# label2id = {'Normal/Mild': 0, 'Moderate':1, 'Severe':2}
# df = df.replace(label2id)
# df.head()

In [24]:
CONDITIONS = [
    'Spinal Canal Stenosis', 
    'Left Neural Foraminal Narrowing', 
    'Right Neural Foraminal Narrowing',
    'Left Subarticular Stenosis',
    'Right Subarticular Stenosis'
]

LEVELS = [
    'L1/L2',
    'L2/L3',
    'L3/L4',
    'L4/L5',
    'L5/S1',
]

In [25]:
class RSNA24Dataset(Dataset):
    def __init__(self, df, phase='train', transform=None):
        self.df = df
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = np.zeros((512, 512, IN_CHANS), dtype=np.uint8)
        t = self.df.iloc[idx]
        st_id = int(t['study_id'])
        label = t[1:].values.astype(np.int64)
        
        # Sagittal T1
        for i in range(0, 10, 1):
            try:
                p = f'./cvt_png/{st_id}/Sagittal T1/{i:03d}.png'
                img = Image.open(p).convert('L')
                img = np.array(img)
                x[..., i] = img.astype(np.uint8)
            except Exception as e:
#                 print(f'Failed to load on {st_id}, Sagittal T1, error: {e}')
#                 traceback.print_exc()
                pass
            
        # Sagittal T2/STIR
        for i in range(0, 10, 1):
            try:
                p = f'./cvt_png/{st_id}/Sagittal T2_STIR/{i:03d}.png'
                img = Image.open(p).convert('L')
                img = np.array(img)
                x[..., i+10] = img.astype(np.uint8)
            except Exception as e:
#                 print(f'Failed to load on {st_id}, Sagittal T1, error: {e}')
#                 traceback.print_exc()
                pass
            
        # Axial T2
        axt2 = glob.glob(f'./cvt_png/{st_id}/Axial T2/*.png')
        axt2 = sorted(axt2)
    
        step = len(axt2) / 10.0
        st = len(axt2)/2.0 - 4.0*step
        end = len(axt2)+0.0001
                
        for i, j in enumerate(np.arange(st, end, step)):
            try:
                p = axt2[max(0, int((j-0.5001).round()))]
                img = Image.open(p).convert('L')
                img = np.array(img)
                x[..., i+20] = img.astype(np.uint8)
            except Exception as e:
#                 print(f'Failed to load on {st_id}, Sagittal T1, error: {e}')
#                 traceback.print_exc()
                pass
            
        assert np.sum(x)>0
            
        if self.transform is not None:
            x = self.transform(image=x)['image']

        x = x.transpose(2, 0, 1)
                
        return x, label

In [26]:
# transforms_train = A.Compose([
#     A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=AUG_PROB),
#     A.OneOf([
#         A.MotionBlur(blur_limit=5),
#         A.MedianBlur(blur_limit=5),
#         A.GaussianBlur(blur_limit=5),
#         A.GaussNoise(var_limit=(5.0, 30.0)),
#     ], p=AUG_PROB),

#     A.OneOf([
#         A.OpticalDistortion(distort_limit=1.0),
#         A.GridDistortion(num_steps=5, distort_limit=1.),
#         A.ElasticTransform(alpha=3),
#     ], p=AUG_PROB),

#     A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),
#     A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
#     A.CoarseDropout(max_holes=16, max_height=64, max_width=64, min_holes=1, min_height=8, min_width=8, p=AUG_PROB),    
#     A.Normalize(mean=0.5, std=0.5)
# ])

# transforms_val = A.Compose([
#     A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
#     A.Normalize(mean=0.5, std=0.5)
# ])

# if not NOT_DEBUG or not AUG:
#     transforms_train = transforms_val

In [27]:
# tmp_ds = RSNA24Dataset(df, phase='train', transform=transforms_train)
# tmp_dl = DataLoader(
#             tmp_ds,
#             batch_size=1,
#             shuffle=False,
#             pin_memory=True,
#             drop_last=False,
#             num_workers=0
#             )


In [28]:

# for i, (x, t) in enumerate(tmp_dl):
#     if i==5:break
#     print('x stat:', x.shape, x.min(), x.max(),x.mean(), x.std())
#     print(t, t.shape)
#     y = x.numpy().transpose(0,2,3,1)[0,...,:3]
#     y = (y + 1) / 2
#     plt.imshow(y)
#     plt.show()
#     print('y stat:', y.shape, y.min(), y.max(),y.mean(), y.std())
#     print()
# plt.close()
# del tmp_ds, tmp_dl

In [29]:
class RSNA24Model(nn.Module):
    def __init__(self, model_name, in_c=30, n_classes=75, pretrained=True, features_only=False):
        super().__init__()
        self.model = timm.create_model(
                                    model_name,
                                    pretrained=pretrained, 
                                    features_only=features_only,
                                    in_chans=in_c,
                                    num_classes=n_classes,
                                    global_pool='avg'
                                    )
    
    def forward(self, x):
        y = self.model(x)
        return y

In [30]:
# m = RSNA24Model(MODEL_NAME, in_c=IN_CHANS, n_classes=N_CLASSES, pretrained=False)
# i = torch.randn(2, IN_CHANS, 512, 512)
# out = m(i)
# for o in out:
#     print(o.shape, o.min(), o.max())

In [31]:
# del m, i, out

In [32]:
skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

In [33]:
# from collections import OrderedDict
# import math
# from torch.utils.data import DataLoader
# from torch.optim import AdamW
# from transformers import get_cosine_schedule_with_warmup

# # Autocast and GradScaler initialization
# autocast = torch.amp.autocast(device, enabled=USE_AMP, dtype=torch.half)
# scaler = torch.amp.GradScaler(enabled=USE_AMP, init_scale=4096)

# # Stratified K-Fold

# for fold, (trn_idx, val_idx) in enumerate(skf.split(range(len(df)))):
#     print('#'*30)
#     print(f'Start fold {fold}')
#     print('#'*30)
#     print(len(trn_idx), len(val_idx))

#     # Data preparation
#     df_train = df.iloc[trn_idx]
#     df_valid = df.iloc[val_idx]

#     train_ds = RSNA24Dataset(df_train, phase='train', transform=transforms_train)
#     train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, drop_last=True, num_workers=N_WORKERS)

#     valid_ds = RSNA24Dataset(df_valid, phase='valid', transform=transforms_val)
#     valid_dl = DataLoader(valid_ds, batch_size=BATCH_SIZE * 2, shuffle=False, pin_memory=True, drop_last=False, num_workers=N_WORKERS)

#     # Model, optimizer, scheduler
#     model = RSNA24Model(MODEL_NAME, IN_CHANS, N_CLASSES, pretrained=True)
#     model.to(device)

#     optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

#     warmup_steps = EPOCHS / 10 * len(train_dl) // GRAD_ACC
#     num_total_steps = EPOCHS * len(train_dl) // GRAD_ACC
#     num_cycles = 0.475
#     scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_total_steps, num_cycles=num_cycles)

#     # Loss function with weights
#     weights = torch.tensor([1.0, 2.0, 4.0])
#     criterion = nn.CrossEntropyLoss(weight=weights.to(device))

#     best_loss = 1.2
#     best_wll = 1.2
#     es_step = 0

#     def compute_loss(y, t, criterion):
#         """Helper function to compute loss over columns."""
#         loss = 0
#         for col in range(N_LABELS):
#             pred = y[:, col * 3: col * 3 + 3]
#             gt = t[:, col]
#             loss += criterion(pred, gt) / N_LABELS
#         return loss

#     # Training and validation loop
#     for epoch in range(1, EPOCHS + 1):
#         print(f'Start epoch {epoch}')

#         # Training phase
#         model.train()
#         total_loss = 0
#         optimizer.zero_grad(set_to_none=True)

#         with tqdm(train_dl, leave=True) as pbar:
#             for idx, (x, t) in enumerate(pbar):
#                 x, t = x.to(device), t.to(device)

#                 with autocast:
#                     y = model(x)
#                     loss = compute_loss(y, t, criterion)

#                     total_loss += loss.item()
#                     if GRAD_ACC > 1:
#                         loss /= GRAD_ACC

#                 # Handle NaN loss
#                 if not math.isfinite(loss):
#                     print(f"Loss is {loss}, stopping training")
#                     sys.exit(1)

#                 pbar.set_postfix(OrderedDict(loss=f'{loss.item() * GRAD_ACC:.6f}', lr=f'{optimizer.param_groups[0]["lr"]:.3e}'))
#                 scaler.scale(loss).backward()

#                 torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM or 1e9)

#                 if (idx + 1) % GRAD_ACC == 0:
#                     scaler.step(optimizer)
#                     scaler.update()
#                     optimizer.zero_grad(set_to_none=True)

#                     if scheduler is not None:
#                         scheduler.step()

#         train_loss = total_loss / len(train_dl)
#         print(f'train_loss: {train_loss:.6f}')

#         # Validation phase
#         model.eval()
#         total_loss = 0
#         y_preds = []
#         labels = []

#         with tqdm(valid_dl, leave=True) as pbar:
#             with torch.no_grad():
#                 for idx, (x, t) in enumerate(pbar):
#                     x, t = x.to(device), t.to(device)

#                     with autocast:
#                         y = model(x)
#                         loss = compute_loss(y, t, criterion)
#                         total_loss += loss.item()

#                         # Collect predictions and labels for evaluation
#                         for col in range(N_LABELS):
#                             pred = y[:, col * 3:col * 3 + 3]
#                             y_preds.append(pred.float().cpu())
#                             labels.append(t[:, col].cpu())

#         val_loss = total_loss / len(valid_dl)
#         y_preds = torch.cat(y_preds, dim=0)
#         labels = torch.cat(labels)
#         val_wll = nn.CrossEntropyLoss()(y_preds, labels)

#         print(f'val_loss: {val_loss:.6f}, val_wll: {val_wll:.6f}')

#         # Early stopping and saving the best model
#         if val_loss < best_loss or val_wll < best_wll:
#             es_step = 0

#             if val_loss < best_loss:
#                 print(f'Epoch: {epoch}, best loss updated from {best_loss:.6f} to {val_loss:.6f}')
#                 best_loss = val_loss

#             if val_wll < best_wll:
#                 print(f'Epoch: {epoch}, best wll_metric updated from {best_wll:.6f} to {val_wll:.6f}')
#                 best_wll = val_wll
#                 torch.save(model.state_dict(), f'{OUTPUT_DIR}/best_wll_model_fold-{fold}.pt')

#         else:
#             es_step += 1
#             if es_step >= EARLY_STOPPING_EPOCH:
#                 print('Early stopping')
#                 break


In [34]:
# cv = 0
# y_preds = []
# labels = []
# weights = torch.tensor([1.0, 2.0, 4.0])
# criterion2 = nn.CrossEntropyLoss(weight=weights)

# for fold, (trn_idx, val_idx) in enumerate(skf.split(range(len(df)))):
#     print('#'*30)
#     print(f'start fold{fold}')
#     print('#'*30)
#     df_valid = df.iloc[val_idx]
#     valid_ds = RSNA24Dataset(df_valid, phase='valid', transform=transforms_val)
#     valid_dl = DataLoader(
#                 valid_ds,
#                 batch_size=1,
#                 shuffle=False,
#                 pin_memory=True,
#                 drop_last=False,
#                 num_workers=N_WORKERS
#                 )

#     model = RSNA24Model(MODEL_NAME, IN_CHANS, N_CLASSES, pretrained=False)    
#     fname = f'{MODEL_DIR}/best_wll_model_fold-{fold}.pt'
#     model.load_state_dict(torch.load(fname))
#     model.to(device)   
    
#     model.eval()
#     with tqdm(valid_dl, leave=True) as pbar:
#         with torch.no_grad():
#             for idx, (x, t) in enumerate(pbar):
                
#                 x = x.to(device)
#                 t = t.to(device)
                    
#                 with autocast:
#                     y = model(x)
#                     for col in range(N_LABELS):
#                         pred = y[:,col*3:col*3+3]
#                         gt = t[:,col] 
#                         y_pred = pred.float()
#                         y_preds.append(y_pred.cpu())
#                         labels.append(gt.cpu())

# y_preds = torch.cat(y_preds)
# labels = torch.cat(labels)

In [35]:
# cv = criterion2(y_preds, labels)
# print('cv score:', cv.item())

In [36]:
# from sklearn.metrics import log_loss
# y_pred_np = y_preds.softmax(1).numpy()
# labels_np = labels.numpy()
# y_pred_nan = np.zeros((y_preds.shape[0], 1))
# y_pred2 = np.concatenate([y_pred_nan, y_pred_np],axis=1)
# weights = []
# for l in labels:
#     if l==0: weights.append(1)
#     elif l==1: weights.append(2)
#     elif l==2: weights.append(4)
#     else: weights.append(0)
# cv2 = log_loss(labels, y_pred2, normalize=True, sample_weight=weights)
# print('cv score:', cv2)

In [37]:
# np.save(f'{OUTPUT_DIR}/labels.npy', labels_np)
# np.save(f'{OUTPUT_DIR}/final_oof.npy', y_pred2)

In [38]:
df = pd.read_csv(f'{rd}/test_series_descriptions.csv')
df.head()

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [39]:
study_ids = list(df['study_id'].unique())

In [40]:
sample_sub = pd.read_csv(f'{rd}/sample_submission.csv')

In [41]:
LABELS = list(sample_sub.columns[1:])
LABELS

['normal_mild', 'moderate', 'severe']

In [42]:
# CONDITIONS = [
#     'spinal_canal_stenosis', 
#     'left_neural_foraminal_narrowing', 
#     'right_neural_foraminal_narrowing',
#     'left_subarticular_stenosis',
#     'right_subarticular_stenosis'
# ]

# LEVELS = [
#     'l1_l2',
#     'l2_l3',
#     'l3_l4',
#     'l4_l5',
#     'l5_s1',
# ]

In [43]:
class RSNA24TestDataset(Dataset):
    def __init__(self, df, study_ids, phase='test', transform=None):
        self.df = df
        self.study_ids = study_ids
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.study_ids)
    
    def get_img_paths(self, study_id, series_desc):
        pdf = self.df[self.df['study_id']==study_id]
        pdf_ = pdf[pdf['series_description']==series_desc]
        allimgs = []
        for i, row in pdf_.iterrows():
            pimgs = glob.glob(f'{rd}/test_images/{study_id}/{row["series_id"]}/*.dcm')
            pimgs = sorted(pimgs, key=natural_keys)
            allimgs.extend(pimgs)
            
        return allimgs
    
    def read_dcm_ret_arr(self, src_path):
        dicom_data = pydicom.dcmread(src_path)
        image = dicom_data.pixel_array
        image = (image - image.min()) / (image.max() - image.min() + 1e-6) * 255
        img = cv2.resize(image, (IMG_SIZE[0], IMG_SIZE[1]),interpolation=cv2.INTER_CUBIC)
        assert img.shape==(IMG_SIZE[0], IMG_SIZE[1])
        return img

    def __getitem__(self, idx):
        x = np.zeros((IMG_SIZE[0], IMG_SIZE[1], IN_CHANS), dtype=np.uint8)
        st_id = self.study_ids[idx]        
        
        # Sagittal T1
        allimgs_st1 = self.get_img_paths(st_id, 'Sagittal T1')
        if len(allimgs_st1)==0:
            print(st_id, ': Sagittal T1, has no images')
        
        else:
            step = len(allimgs_st1) / 10.0
            st = len(allimgs_st1)/2.0 - 4.0*step
            end = len(allimgs_st1)+0.0001
            for j, i in enumerate(np.arange(st, end, step)):
                try:
                    ind2 = max(0, int((i-0.5001).round()))
                    img = self.read_dcm_ret_arr(allimgs_st1[ind2])
                    x[..., j] = img.astype(np.uint8)
                except:
                    print(f'failed to load on {st_id}, Sagittal T1')
                    pass
            
        # Sagittal T2/STIR
        allimgs_st2 = self.get_img_paths(st_id, 'Sagittal T2/STIR')
        if len(allimgs_st2)==0:
            print(st_id, ': Sagittal T2/STIR, has no images')
            
        else:
            step = len(allimgs_st2) / 10.0
            st = len(allimgs_st2)/2.0 - 4.0*step
            end = len(allimgs_st2)+0.0001
            for j, i in enumerate(np.arange(st, end, step)):
                try:
                    ind2 = max(0, int((i-0.5001).round()))
                    img = self.read_dcm_ret_arr(allimgs_st2[ind2])
                    x[..., j+10] = img.astype(np.uint8)
                except:
                    print(f'failed to load on {st_id}, Sagittal T2/STIR')
                    pass
            
        # Axial T2
        allimgs_at2 = self.get_img_paths(st_id, 'Axial T2')
        if len(allimgs_at2)==0:
            print(st_id, ': Axial T2, has no images')
            
        else:
            step = len(allimgs_at2) / 10.0
            st = len(allimgs_at2)/2.0 - 4.0*step
            end = len(allimgs_at2)+0.0001

            for j, i in enumerate(np.arange(st, end, step)):
                try:
                    ind2 = max(0, int((i-0.5001).round()))
                    img = self.read_dcm_ret_arr(allimgs_at2[ind2])
                    x[..., j+20] = img.astype(np.uint8)
                except:
                    print(f'failed to load on {st_id}, Axial T2')
                    pass  
            
            
        if self.transform is not None:
            x = self.transform(image=x)['image']

        x = x.transpose(2, 0, 1)
                
        return x, str(st_id)

In [44]:
transforms_test = A.Compose([
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.Normalize(mean=0.5, std=0.5)
])

In [45]:
test_ds = RSNA24TestDataset(df, study_ids, transform=transforms_test)
test_dl = DataLoader(
    test_ds, 
    batch_size=1, 
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=True,
    drop_last=False
)

In [46]:
# class RSNA24Model(nn.Module):
#     def __init__(self, model_name, in_c=30, n_classes=75, pretrained=True, features_only=False):
#         super().__init__()
#         self.model = timm.create_model(
#                                     model_name,
#                                     pretrained=pretrained, 
#                                     features_only=features_only,
#                                     in_chans=in_c,
#                                     num_classes=n_classes,
#                                     global_pool='avg'
#                                     )
    
#     def forward(self, x):
#         y = self.model(x)
#         return y

In [47]:
models = []

In [48]:
CKPT_PATHS = glob.glob('/kaggle/input/test/pytorch/default/2/best_wll_model_fold-*.pt')
CKPT_PATHS = sorted(CKPT_PATHS)

In [49]:
for i, cp in enumerate(CKPT_PATHS):
    print(f'loading {cp}...')
    model = RSNA24Model(MODEL_NAME, IN_CHANS, N_CLASSES, pretrained=False)
    model.load_state_dict(torch.load(cp))
    model.eval()
    model.half()
    model.to(device)
    models.append(model)

loading /kaggle/input/test/pytorch/default/2/best_wll_model_fold-0.pt...


/tmp/ipykernel_24/1182973946.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(cp))


loading /kaggle/input/test/pytorch/default/2/best_wll_model_fold-1.pt...
loading /kaggle/input/test/pytorch/default/2/best_wll_model_fold-2.pt...
loading /kaggle/input/test/pytorch/default/2/best_wll_model_fold-3.pt...
loading /kaggle/input/test/pytorch/default/2/best_wll_model_fold-4.pt...


In [50]:
autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half)
y_preds = []
row_names = []

with tqdm(test_dl, leave=True) as pbar:
    with torch.no_grad():
        for idx, (x, si) in enumerate(pbar):
            x = x.to(device)
            pred_per_study = np.zeros((25, 3))
            
            for cond in CONDITIONS:
                for level in LEVELS:
                    row_names.append(si[0] + '_' + cond + '_' + level)
            
            with autocast:
                for m in models:
                    y = m(x)[0]
                    for col in range(N_LABELS):
                        pred = y[col*3:col*3+3]
                        y_pred = pred.float().softmax(0).cpu().numpy()
                        pred_per_study[col] += y_pred / len(models)
                y_preds.append(pred_per_study)

y_preds = np.concatenate(y_preds, axis=0)

/tmp/ipykernel_24/1318220155.py:1: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half)
100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


In [51]:
sub = pd.DataFrame()
sub['row_id'] = row_names
sub[LABELS] = y_preds
sub.head(25)

,row_id,normal_mild,moderate,severe
0,44036939_Spinal Canal Stenosis_L1/L2,0.575941,0.286909,0.137150
1,44036939_Spinal Canal Stenosis_L2/L3,0.278065,0.387061,0.334874
2,44036939_Spinal Canal Stenosis_L3/L4,0.173202,0.317942,0.508856
3,44036939_Spinal Canal Stenosis_L4/L5,0.252347,0.190865,0.556788
4,44036939_Spinal Canal Stenosis_L5/S1,0.828362,0.084823,0.086815
5,44036939_Left Neural Foraminal Narrowing_L1/L2,0.671752,0.298019,0.030229
6,44036939_Left Neural Foraminal Narrowing_L2/L3,0.361944,0.551196,0.086861
7,44036939_Left Neural Foraminal Narrowing_L3/L4,0.288582,0.494857,0.216561
8,44036939_Left Neural Foraminal Narrowing_L4/L5,0.129308,0.502899,0.367793
9,44036939_Left Neural Foraminal Narrowing_L5/S1,0.161376,0.359462,0.479162


In [52]:
sub.to_csv('submission.csv', index=False)